In [1]:
Experiments_file='/home/kate/Research/YearBuilt/Experiments/DwellingExperiments.xlsx'
AllExperiments_tab='Experiments'
#Experiment name will be used as Hyperparameter job name
Experiment_name='PropertyHyperParams'
#1. Experiment configuration: the same dataset is used in all trial omponents
Experiment_InputData='%s InputData'%Experiment_name
#2. HyperParam configuration: min, max value and type: ContinuousParameterRanges, IntegerParameterRanges
#If MinValue=MaxValue it's a static parameter and will not be added into tuning job but into models directly
Experiment_models='%s Ranges'%Experiment_name

Trial_name='%s-TrainingModels'%Experiment_name 

objective_metric_name = 'validation:auc'

#model in
bucket='kdproperty'

#instance type to be created for training jobs. 
instance_type='ml.c5.xlarge'
#Script to be load in training instance
training_job_entry_point='ModelTraining.py'
#data in training and testing file type
content_type='text/csv'

In [2]:
#packages used in the notebook
import time
import sys
import boto3
import sagemaker
from sagemaker.session import s3_input
from sagemaker.xgboost.estimator import XGBoost
from sagemaker.xgboost.model import XGBoostModel
import pandas as pd
import numpy as np
from smexperiments.experiment import Experiment
from smexperiments.trial import Trial
from sagemaker.analytics import ExperimentAnalytics
from sagemaker import HyperparameterTuningJobAnalytics, Session
from datetime import datetime, timezone
from smexperiments.search_expression import Filter, Operator, SearchExpression
from smexperiments.trial_component import TrialComponent
from sagemaker.tuner import IntegerParameter
from sagemaker.tuner import ContinuousParameter
from sagemaker.tuner import HyperparameterTuner

In [3]:
#should be run as a first step
#role arn is used when run from a local machine
role = 'arn:aws:iam::757107622481:role/service-role/AmazonSageMaker-ExecutionRole-20200819T131882'
region = 'us-west-2'
sagemaker_session = sagemaker.session.Session(default_bucket=bucket)
sm = boto3.Session().client('sagemaker')

INFO:botocore.credentials:Found credentials in shared credentials file: ~/.aws/credentials
INFO:botocore.credentials:Found credentials in shared credentials file: ~/.aws/credentials


In [4]:
sys.path.append('/home/kate/Research/YearBuilt/Notebooks/Property')
import ExperimentsUtils as eu

In [5]:
experiments = pd.read_excel(open(Experiments_file, 'rb'), sheet_name=AllExperiments_tab)

In [6]:
#delete experiment and trials but not output files associated with jobs in experiments
eu.cleanup_experiment(Experiment_name)
#eu.cleanup_trial(Experiment_name, Trial_name)

INFO:botocore.credentials:Found credentials in shared credentials file: ~/.aws/credentials
INFO:botocore.credentials:Found credentials in shared credentials file: ~/.aws/credentials
INFO:botocore.credentials:Found credentials in shared credentials file: ~/.aws/credentials
INFO:botocore.credentials:Found credentials in shared credentials file: ~/.aws/credentials
INFO:botocore.credentials:Found credentials in shared credentials file: ~/.aws/credentials
INFO:botocore.credentials:Found credentials in shared credentials file: ~/.aws/credentials
INFO:botocore.credentials:Found credentials in shared credentials file: ~/.aws/credentials
INFO:botocore.credentials:Found credentials in shared credentials file: ~/.aws/credentials
INFO:botocore.credentials:Found credentials in shared credentials file: ~/.aws/credentials
INFO:botocore.credentials:Found credentials in shared credentials file: ~/.aws/credentials
INFO:botocore.credentials:Found credentials in shared credentials file: ~/.aws/credentials

In [7]:
eu.create_experiment(Experiment_name)
eu.create_trial(Experiment_name,Trial_name)

INFO:botocore.credentials:Found credentials in shared credentials file: ~/.aws/credentials
INFO:botocore.credentials:Found credentials in shared credentials file: ~/.aws/credentials
INFO:botocore.credentials:Found credentials in shared credentials file: ~/.aws/credentials
INFO:botocore.credentials:Found credentials in shared credentials file: ~/.aws/credentials


In [8]:
#The same input data for all models, created in an other experiment
InputData = pd.read_excel(open(Experiments_file, 'rb'), sheet_name=Experiment_InputData)
InputData

,Model,Trial Component,Training_data,Validation_data
0,Dwelling,Dwelling-PreparingTrainValidData-Dwelling,s3://kdproperty/Data/training_data/Dwelling,s3://kdproperty/Data/validation_data/Dwelling


In [9]:
train_input = s3_input(InputData['Training_data'][0], content_type=content_type)
print('Train dataset: %s'%train_input)
validation_input = s3_input(InputData['Validation_data'][0], content_type=content_type)
print('Validation dataset: %s'%validation_input)

See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.


Train dataset: <sagemaker.deprecations.deprecated_class.<locals>.DeprecatedClass object at 0x7fd07cfee320>
Validation dataset: <sagemaker.deprecations.deprecated_class.<locals>.DeprecatedClass object at 0x7fd07cfee080>


In [10]:
tuning_job_ranges = pd.read_excel(open(Experiments_file, 'rb'), sheet_name=Experiment_models)
tuning_job_ranges

,Name,RangeType,MinValue,MaxValue,scaling_type
0,eta,ContinuousParameter,0.01,0.1,Auto
1,colsample_bylevel,ContinuousParameter,0.00,0.8,Auto
2,colsample_bytree,ContinuousParameter,0.00,0.8,Auto
3,scale_pos_weight,ContinuousParameter,0.30,0.3,Auto
4,max_depth,IntegerParameter,5.00,5.0,Auto
5,num_round,IntegerParameter,5000.00,10000.0,Auto


In [11]:
#only non static params into tuning job config
tuning_job_config = dict()
for index, row in tuning_job_ranges[tuning_job_ranges['MinValue']!=tuning_job_ranges['MaxValue']].iterrows():
    if row['RangeType']=='ContinuousParameter':
        tuning_job_config[row['Name']]=ContinuousParameter(row['MinValue'], row['MaxValue'],scaling_type=row['scaling_type'])
    elif row['RangeType']=='IntegerParameter':
        tuning_job_config[row['Name']]=IntegerParameter(int(row['MinValue']), int(row['MaxValue']),scaling_type=row['scaling_type'])
tuning_job_config

{'eta': <sagemaker.parameter.ContinuousParameter at 0x7fd07f1b6b70>,
 'colsample_bylevel': <sagemaker.parameter.ContinuousParameter at 0x7fd07f1b6a58>,
 'colsample_bytree': <sagemaker.parameter.ContinuousParameter at 0x7fd07f1b6c50>,
 'num_round': <sagemaker.parameter.IntegerParameter at 0x7fd07f1b6b38>}

In [12]:
#static base XGB parameters
base_params = {'objective': 'binary:logistic', 'eval_metric': 'auc', 'silent': True,'booster': 'gbtree','seed':42}

In [13]:
#only non static params into model XGB parameters
parameterset = dict()
for index, row in tuning_job_ranges[tuning_job_ranges['MinValue']==tuning_job_ranges['MaxValue']].iterrows():
    if row['RangeType']=='IntegerParameter':
        parameterset[row['Name']]=str(int(row['MaxValue']))
    else:
        parameterset[row['Name']]=str(row['MaxValue'])       
   
    
xgb_params = {**base_params, **parameterset}
xgb_params

{'objective': 'binary:logistic',
 'eval_metric': 'auc',
 'silent': True,
 'booster': 'gbtree',
 'seed': 42,
 'scale_pos_weight': '0.3',
 'max_depth': '5'}

In [14]:
%%writefile ModelTraining.py

#  Copyright 2018 Amazon.com, Inc. or its affiliates. All Rights Reserved.
#
#  Licensed under the Apache License, Version 2.0 (the "License").
#  You may not use this file except in compliance with the License.
#  A copy of the License is located at
#
#      http://www.apache.org/licenses/LICENSE-2.0
#
#  or in the "license" file accompanying this file. This file is distributed
#  on an "AS IS" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either
#  express or implied. See the License for the specific language governing
#  permissions and limitations under the License.
from __future__ import print_function

import argparse
import json
import logging
import os
import pandas as pd
import pickle as pkl

from sagemaker_containers import entry_point
from sagemaker_xgboost_container.data_utils import get_dmatrix
from sagemaker_xgboost_container import distributed

import xgboost as xgb

import pandas as pd
import numpy as np


def _xgb_train(params, dtrain, evals, num_boost_round, model_dir, is_master):
    """Run xgb train on arguments given with rabit initialized.

    This is our rabit execution function.

    :param args_dict: Argument dictionary used to run xgb.train().
    :param is_master: True if current node is master host in distributed training,
                        or is running single node training job.
                        Note that rabit_run will include this argument.
    """
    booster = xgb.train(params=params,
                        dtrain=dtrain,
                        evals=evals,
                        maximize=True,
                        num_boost_round=num_boost_round,
                        early_stopping_rounds=100,
                        verbose_eval=100)

    if is_master:
        model_location = model_dir + '/xgboost-model'
        pkl.dump(booster, open(model_location, 'wb'))
        logging.info("Stored trained model at {}".format(model_location))


if __name__ == '__main__':
    parser = argparse.ArgumentParser()

    # Hyperparameters are described here.
    parser.add_argument('--max_depth', type=int,)
    parser.add_argument('--eta', type=float)
    parser.add_argument('--objective', type=str)
    parser.add_argument('--num_round', type=int)
    
    parser.add_argument('--eval_metric', type=str)
    parser.add_argument('--seed', type=float)
    parser.add_argument('--scale_pos_weight', type=float)
    parser.add_argument('--colsample_bylevel', type=float)
    parser.add_argument('--colsample_bytree', type=float)
        
            
            

    # Sagemaker specific arguments. Defaults are set in the environment variables.
    parser.add_argument('--output_data_dir', type=str, default=os.environ.get('SM_OUTPUT_DATA_DIR'))
    parser.add_argument('--model_dir', type=str, default=os.environ.get('SM_MODEL_DIR'))
    parser.add_argument('--train', type=str, default=os.environ.get('SM_CHANNEL_TRAIN'))
    parser.add_argument('--validation', type=str, default=os.environ.get('SM_CHANNEL_VALIDATION'))
    parser.add_argument('--sm_hosts', type=str, default=os.environ.get('SM_HOSTS'))
    parser.add_argument('--sm_current_host', type=str, default=os.environ.get('SM_CURRENT_HOST'))

    args, _ = parser.parse_known_args()

    # Get SageMaker host information from runtime environment variables
    sm_hosts = json.loads(args.sm_hosts)
    sm_current_host = args.sm_current_host

    dtrain = get_dmatrix(args.train, 'csv')
    dval = get_dmatrix(args.validation, 'csv')
    watchlist = [(dtrain, 'train'), (dval, 'validation')] if dval is not None else [(dtrain, 'train')]

    
    
    train_hp = {
        'max_depth': args.max_depth,
        'eta': args.eta,
        'objective': args.objective,
        'silent': True,
        'booster': 'gbtree',
        'seed': 42,
        'eval_metric':args.eval_metric,
        'scale_pos_weight':args.scale_pos_weight,
        'colsample_bylevel': args.colsample_bylevel,
        'colsample_bytree': args.colsample_bytree
        }

    xgb_train_args = dict(
        params=train_hp,
        dtrain=dtrain,
        evals=watchlist,
        num_boost_round=args.num_round,
        model_dir=args.model_dir)

    if len(sm_hosts) > 1:
        # Wait until all hosts are able to find each other
        entry_point._wait_hostname_resolution()

        # Execute training function after initializing rabit.
        distributed.rabit_run(
            exec_fun=_xgb_train,
            args=xgb_train_args,
            include_in_training=(dtrain is not None),
            hosts=sm_hosts,
            current_host=sm_current_host,
            update_rabit_args=True
        )
    else:
        # If single node training, call training method directly.
        if dtrain:
            xgb_train_args['is_master'] = True
            _xgb_train(**xgb_train_args)
        else:
            raise ValueError("Training channel must have data to train model.")


def model_fn(model_dir):
    """Deserialize and return fitted model.

    Note that this should have the same name as the serialized model in the _xgb_train method
    """
    model_file = 'xgboost-model'
    booster = pkl.load(open(os.path.join(model_dir, model_file), 'rb'))
    return booster

Overwriting ModelTraining.py


In [15]:
xgb_script_mode_estimator = XGBoost(
        entry_point=training_job_entry_point,
        hyperparameters=xgb_params,
        role=role, 
        train_instance_count=1,
        train_instance_type=instance_type,
        framework_version="1.0-1",
        output_path='s3://%s/%s'%(bucket,Experiment_name)
        )

See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
INFO:botocore.credentials:Found credentials in shared credentials file: ~/.aws/credentials


In [16]:
tuner = HyperparameterTuner(
    estimator=xgb_script_mode_estimator,
    objective_metric_name=objective_metric_name,
    hyperparameter_ranges =tuning_job_config,
    max_jobs=100,
    max_parallel_jobs=10,
    strategy='Bayesian',
    base_tuning_job_name=Experiment_name
)

In [17]:
tuner.fit({'train': train_input, 'validation': validation_input}, include_cls_metadata=False,wait=False)

INFO:sagemaker.image_uris:Defaulting to the only supported framework/algorithm version: latest.
INFO:sagemaker.image_uris:Ignoring unnecessary instance type: None.
INFO:sagemaker:Creating hyperparameter tuning job with name: PropertyHyperParams-201230-1714


In [18]:
#Associating training jobs created from the tuning job with an experiment if needed

In [19]:
# the training job names contain the tuning job name (and the training job name is in the source arn)
source_arn_filter = Filter(name="Source.SourceArn", operator=Operator.CONTAINS, value=Experiment_name.lower())
source_type_filter = Filter(name="Source.SourceType", operator=Operator.EQUALS, value="SageMakerTrainingJob")

search_expression = SearchExpression(filters=[source_arn_filter, source_type_filter])

In [20]:
HyperParameterTuningJobName=tuner.latest_tuning_job.job_name
job_desc = sm.describe_hyper_parameter_tuning_job(HyperParameterTuningJobName=HyperParameterTuningJobName)
job_desc['HyperParameterTuningJobStatus']

'InProgress'

In [21]:
#Hyperparameter tuning job is not associated with an experiment directly
#if needed we can do it via training jobs
#we need trial object, created above to join jobs to it
try:
    trial = Trial.load(trial_name=Trial_name)
except Exception as ex:
    if "ResourceNotFound" in str(ex):
        trial = Trial.create(experiment_name=Experiment_name, trial_name=Trial_name)

INFO:botocore.credentials:Found credentials in shared credentials file: ~/.aws/credentials


In [23]:
#while tuning job is still in progress
job_desc = sm.describe_hyper_parameter_tuning_job(HyperParameterTuningJobName=HyperParameterTuningJobName)
#wait time in  sec
wait_time=600
while job_desc['HyperParameterTuningJobStatus']=='InProgress':
    trial_component_search_results = list(TrialComponent.search(search_expression=search_expression, sagemaker_boto_client=sm))
    # associate the trial components with the trial
    for tc in trial_component_search_results:
        trial.add_trial_component(tc.trial_component_name)
        print('Added %s'%tc.trial_component_name)
        # sleep to avoid throttling
        time.sleep(1)
    time.sleep(wait_time)   
    job_desc = sm.describe_hyper_parameter_tuning_job(HyperParameterTuningJobName=HyperParameterTuningJobName)

In [24]:
trial_component_analytics = ExperimentAnalytics(experiment_name=Experiment_name)
trial_comp_ds = trial_component_analytics.dataframe()
trial_ds=trial_comp_ds.copy()

INFO:botocore.credentials:Found credentials in shared credentials file: ~/.aws/credentials


In [25]:
#hyperparameters used in models
#should be adjusted for the parameters used in tuning
ModelsParams = trial_ds[['DisplayName',
                                                                      'objective', 
                                                                      'eval_metric', 
                                                                      'scale_pos_weight', 
                                                                      'colsample_bylevel',
                                                                      'colsample_bytree',
                                                                      'eta',
                                                                      'max_depth', 
                                                                      'num_round']]
ModelsParams.columns=['Trial Component',
                                                                      'objective', 
                                                                      'eval_metric',   
                                                                      'scale_pos_weight', 
                                                                      'colsample_bylevel',
                                                                      'colsample_bytree',
                                                                      'eta',
                                                                      'max_depth', 
                                                                      'num_round']
ModelsParams

,Trial Component,objective,eval_metric,scale_pos_weight,colsample_bylevel,colsample_bytree,eta,max_depth,num_round
0,PropertyHyperParams-201230-1714-073-2bb0082d-a...,"""binary:logistic""","""auc""","""0.3""",0.082573,0.620847,0.022982,"""5""",6970.0
1,PropertyHyperParams-201230-1714-079-5fa5c0f7-a...,"""binary:logistic""","""auc""","""0.3""",0.051982,0.619846,0.048555,"""5""",9242.0
2,PropertyHyperParams-201230-1714-078-f9f23bb6-a...,"""binary:logistic""","""auc""","""0.3""",0.000078,0.510618,0.029695,"""5""",9733.0
3,PropertyHyperParams-201230-1714-081-9397142b-a...,"""binary:logistic""","""auc""","""0.3""",0.740077,0.467659,0.041427,"""5""",7895.0
4,PropertyHyperParams-201230-1714-080-fd49715d-a...,"""binary:logistic""","""auc""","""0.3""",0.740077,0.470538,0.041224,"""5""",7895.0
...,...,...,...,...,...,...,...,...,...
98,PropertyHyperParams-201230-1707-004-aaeee7a1-a...,"""binary:logistic""","""auc""","""0.3""",0.068459,0.020560,0.233108,"""5""",6853.0
99,PropertyHyperParams-201230-1707-002-035c8886-a...,"""binary:logistic""","""auc""","""0.3""",0.292587,0.002288,0.869805,"""5""",9591.0
100,PropertyHyperParams-201230-1707-003-ea2a77d2-a...,"""binary:logistic""","""auc""","""0.3""",0.172013,0.606363,0.450447,"""5""",7309.0
101,PropertyHyperParams-201230-1640-041-26e00a43-a...,"""binary:logistic""","""auc""","""0.3""",0.003514,0.003028,0.005869,"""5""",5506.0


In [26]:
#Saving into the Experiment log file models hyperparameters used
eu.SaveToExperimentLog(Experiments_file, '%s ModelParams'%Experiment_name, ModelsParams)

In [27]:
ModelsResults = trial_ds[['DisplayName','train:auc - Last','validation:auc - Last']].sort_values('validation:auc - Last', ascending=False)
ModelsResults.columns=['Trial Component','train:auc','validation:auc']
ModelsResults

,Trial Component,train:auc,validation:auc
59,PropertyHyperParams-201230-1714-021-5425bae0-a...,0.69796,0.65033
50,PropertyHyperParams-201230-1714-031-dffd9a18-a...,0.69796,0.65033
57,PropertyHyperParams-201230-1714-027-4a77046d-a...,0.71238,0.64975
42,PropertyHyperParams-201230-1714-039-66d85aca-a...,0.69121,0.64967
43,PropertyHyperParams-201230-1714-038-2b6718ec-a...,0.69121,0.64967
...,...,...,...
86,PropertyHyperParams-201230-1707-014-af30b3f9-a...,NaN,NaN
87,PropertyHyperParams-201230-1707-017-4e596e55-a...,NaN,NaN
88,PropertyHyperParams-201230-1707-018-a9249a81-a...,NaN,NaN
89,PropertyHyperParams-201230-1707-019-dd669bdc-a...,NaN,NaN


In [28]:
#Saving into the Experiment log file models results
eu.SaveToExperimentLog(Experiments_file, '%s ModelResults'%Experiment_name, ModelsResults)

In [29]:
ModelsResults = trial_ds[['DisplayName','ObjectiveMetric - Min', 'ObjectiveMetric - Max',
       'ObjectiveMetric - Avg', 'ObjectiveMetric - StdDev',
       'ObjectiveMetric - Last', 'ObjectiveMetric - Count']].sort_values('ObjectiveMetric - Last', ascending=False)
ModelsResults

,DisplayName,ObjectiveMetric - Min,ObjectiveMetric - Max,ObjectiveMetric - Avg,ObjectiveMetric - StdDev,ObjectiveMetric - Last,ObjectiveMetric - Count
59,PropertyHyperParams-201230-1714-021-5425bae0-a...,0.5,0.65033,0.610880,0.073972,0.65033,4.0
50,PropertyHyperParams-201230-1714-031-dffd9a18-a...,0.5,0.65033,0.610880,0.073972,0.65033,4.0
57,PropertyHyperParams-201230-1714-027-4a77046d-a...,0.5,0.64975,0.611487,0.074334,0.64975,4.0
42,PropertyHyperParams-201230-1714-039-66d85aca-a...,0.5,0.64967,0.609322,0.073084,0.64967,4.0
43,PropertyHyperParams-201230-1714-038-2b6718ec-a...,0.5,0.64967,0.609322,0.073084,0.64967,4.0
...,...,...,...,...,...,...,...
86,PropertyHyperParams-201230-1707-014-af30b3f9-a...,NaN,NaN,NaN,NaN,NaN,NaN
87,PropertyHyperParams-201230-1707-017-4e596e55-a...,NaN,NaN,NaN,NaN,NaN,NaN
88,PropertyHyperParams-201230-1707-018-a9249a81-a...,NaN,NaN,NaN,NaN,NaN,NaN
89,PropertyHyperParams-201230-1707-019-dd669bdc-a...,NaN,NaN,NaN,NaN,NaN,NaN


In [30]:
#Saving into the Experiment log file models results
eu.SaveToExperimentLog(Experiments_file, '%s ObjectiveMetric'%Experiment_name, ModelsResults)

In [31]:
analytics = HyperparameterTuningJobAnalytics(HyperParameterTuningJobName)
HyperparameterTuning_ds = analytics.dataframe().sort_values(['FinalObjectiveValue'], ascending=False)
HyperparameterTuning_ds

INFO:botocore.credentials:Found credentials in shared credentials file: ~/.aws/credentials


,colsample_bylevel,colsample_bytree,eta,num_round,TrainingJobName,TrainingJobStatus,FinalObjectiveValue,TrainingStartTime,TrainingEndTime,TrainingElapsedTimeSeconds
79,0.657500,0.429352,0.097417,9431.0,PropertyHyperParams-201230-1714-021-5425bae0,Completed,0.65033,2020-12-30 17:24:52-08:00,2020-12-30 17:26:38-08:00,106.0
69,0.657500,0.429352,0.097417,9531.0,PropertyHyperParams-201230-1714-031-dffd9a18,Completed,0.65033,2020-12-30 17:28:21-08:00,2020-12-30 17:29:53-08:00,92.0
73,0.434828,0.666868,0.089638,7016.0,PropertyHyperParams-201230-1714-027-4a77046d,Completed,0.64975,2020-12-30 17:25:06-08:00,2020-12-30 17:26:55-08:00,109.0
62,0.335014,0.396745,0.079259,9995.0,PropertyHyperParams-201230-1714-038-2b6718ec,Completed,0.64967,2020-12-30 17:30:41-08:00,2020-12-30 17:32:20-08:00,99.0
68,0.335014,0.400316,0.079259,9995.0,PropertyHyperParams-201230-1714-032-2132910a,Completed,0.64967,2020-12-30 17:28:34-08:00,2020-12-30 17:30:10-08:00,96.0
...,...,...,...,...,...,...,...,...,...,...
14,0.002861,0.161530,0.025033,7605.0,PropertyHyperParams-201230-1714-086-25eec38f,Completed,0.63442,2020-12-30 17:52:54-08:00,2020-12-30 17:54:43-08:00,109.0
87,0.193046,0.008649,0.026269,9931.0,PropertyHyperParams-201230-1714-013-cc00e51a,Completed,0.63428,2020-12-30 17:21:13-08:00,2020-12-30 17:22:56-08:00,103.0
42,0.677552,0.652838,0.010652,7053.0,PropertyHyperParams-201230-1714-058-c9ffc089,Completed,0.50000,2020-12-30 17:37:54-08:00,2020-12-30 17:38:52-08:00,58.0
40,0.018885,0.644660,0.014077,5144.0,PropertyHyperParams-201230-1714-060-e7bd0ee7,Completed,0.50000,2020-12-30 17:39:26-08:00,2020-12-30 17:40:23-08:00,57.0


In [32]:
#Saving into the Experiment log file models results
eu.SaveToExperimentLog(Experiments_file, '%s FinalObjective'%Experiment_name, ModelsResults)

In [ ]:
#Some visualizations 
#from https://github.com/aws/amazon-sagemaker-examples/blob/master/hyperparameter_tuning/analyze_results/HPO_Analyze_TuningJob_Results.ipynb
#but they do not have a lot of sense for my data
#since parameters are changed and impact scores all together
#maybe 3D area charts makes more sense?

In [33]:
import bokeh
import bokeh.io
bokeh.io.output_notebook()
from bokeh.plotting import figure, show
from bokeh.models import HoverTool

class HoverHelper():

    def __init__(self, tuning_analytics):
        self.tuner = tuning_analytics

    def hovertool(self):
        tooltips = [
            ("FinalObjectiveValue", "@FinalObjectiveValue"),
            ("TrainingJobName", "@TrainingJobName"),
        ]
        for k in self.tuner.tuning_ranges.keys():
            tooltips.append( (k, "@{%s}" % k) )

        ht = HoverTool(tooltips=tooltips)
        return ht

    def tools(self, standard_tools='pan,crosshair,wheel_zoom,zoom_in,zoom_out,undo,reset'):
        return [self.hovertool(), standard_tools]

Loading BokehJS ...

In [34]:
hover = HoverHelper(analytics)

p = figure(plot_width=900, plot_height=400, tools=hover.tools(), x_axis_type='datetime')
p.circle(source=HyperparameterTuning_ds, x='TrainingStartTime', y='FinalObjectiveValue')
show(p)

In [35]:
ranges = analytics.tuning_ranges
figures = []
for hp_name, hp_range in ranges.items():
    categorical_args = {}
    if hp_range.get('Values'):
        # This is marked as categorical.  Check if all options are actually numbers.
        def is_num(x):
            try:
                float(x)
                return 1
            except:
                return 0           
        vals = hp_range['Values']
        if sum([is_num(x) for x in vals]) == len(vals):
            # Bokeh has issues plotting a "categorical" range that's actually numeric, so plot as numeric
            print("Hyperparameter %s is tuned as categorical, but all values are numeric" % hp_name)
        else:
            # Set up extra options for plotting categoricals.  A bit tricky when they're actually numbers.
            categorical_args['x_range'] = vals

    # Now plot it
    p = figure(plot_width=500, plot_height=500, 
               title="Objective vs %s" % hp_name,
               tools=hover.tools(),
               x_axis_label=hp_name, y_axis_label=objective_metric_name,
               **categorical_args)
    p.circle(source=HyperparameterTuning_ds, x=hp_name, y='FinalObjectiveValue')
    figures.append(p)
show(bokeh.layouts.Column(*figures))